In [1]:
from langgraph.graph import StateGraph, END
#from langgraph.prebuilt import ToolExecutor, chat_agent_executor
from langchain_core.runnables import Runnable
from langchain_core.messages import HumanMessage, AnyMessage, SystemMessage, ToolMessage, AIMessage

from typing import TypedDict, List, Union, Annotated
from langchain.agents import initialize_agent, Tool
from langchain_ollama import ChatOllama
from langchain.agents.agent_types import AgentType
import operator

import fastf1
import matplotlib.pyplot as plt
import json
from IPython.display import Image, display


%matplotlib inline

In [2]:
# Modified compare_driver_laps function
def compare_driver_laps(year: int, gp: str, session_type: str, driver1: str, driver2: str):
    session = fastf1.get_session(year, gp, session_type)
    session.load()

    lap1 = session.laps.pick_drivers(driver1).pick_fastest()
    lap2 = session.laps.pick_drivers(driver2).pick_fastest()

    tel1 = lap1.get_car_data().add_distance()
    tel2 = lap2.get_car_data().add_distance()

    # Plot
    plt.figure()
    plt.plot(tel1['Distance'], tel1['Speed'], label=driver1)
    plt.plot(tel2['Distance'], tel2['Speed'], label=driver2)
    plt.xlabel('Distance (m)')
    plt.ylabel('Speed (km/h)')
    plt.title(f'{driver1} vs {driver2} - Fastest Lap Speed Comparison')
    plt.legend()
    plt.grid(True)
    image_path = "comparison.png"
    plt.savefig(image_path)
    plt.close()  # Important to close the figure
    
    # Return both the image path and lap times
    return {
        "image_path": image_path,
        "lap_times": {
            driver1: str(lap1['LapTime']),
            driver2: str(lap2['LapTime'])
        }
    }

In [3]:
# Cell 3: Wrap this function as a LangChain tool

#tool = Tool(
#    name="CompareDriverLaps",
#    func=lambda x: compare_driver_laps(**eval(x)),
#    description="Use this tool to generate a lap speed comparison plot between two drivers. Input should be a dictionary with keys: year, gp, session_type, driver1, driver2."
#)
#print(type(tool))
#print(tool.name)

# Cell 4: LangChain Tool wrapper
tool = Tool(
    name="CompareDriverLaps",
    func=lambda x: compare_driver_laps(**json.loads(x) if isinstance(x, str) else x),
    description=(
        "Use this tool to generate a lap speed comparison plot between two drivers. "
        "Input must be a dictionary with keys: year, gp, session_type, driver1, driver2. "
        "Example: {\"year\": 2023, \"gp\": \"Monaco\", \"session_type\": \"Q\", \"driver1\": \"VER\", \"driver2\": \"LEC\"}"
    )
)
print(type(tool))
print(tool.name)

<class 'langchain_core.tools.simple.Tool'>
CompareDriverLaps


In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [5]:
class Agent:
    
    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_ollama)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
            
        )
        graph.add_edge("action","llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)
    
    
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0
    
    
    def call_ollama(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}    
    
    # Modified take_action method in the Agent class
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:
                print("\n ....bad tool name....")
                result = "bad tool name, retry"
            else:
                result = self.tools[t['name']].invoke(t['args'])
                # If the result contains an image path, display it
                if isinstance(result, dict) and "image_path" in result:
                    display(Image(result["image_path"]))
                    result = f"Comparison complete. Lap times: {result['lap_times']}"
        
            results.append(ToolMessage(
                tool_call_id=t['id'],
                name=t['name'],
                content=str(result)
            ))
        print("Back to the model!")
        return {'messages': results}

In [9]:
# Create and test the agent with the modified code
prompt = """You are a smart F1 assistant.
You can compare drivers using telemetry and lap times. call tools to help.
"""

model = ChatOllama(model="llama3.1")
abot = Agent(model=model, tools=[tool], system=prompt)

messages = [
    HumanMessage(content="Compare Max Verstappen and Charles Leclerc in the 2023 Monaco GP qualifying. Also tell me who was faster.")
]

result = abot.graph.invoke({"messages": messages})

In [10]:
result

{'messages': [HumanMessage(content='Compare Max Verstappen and Charles Leclerc in the 2023 Monaco GP qualifying. Also tell me who was faster.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='{"name": "CompareDriverLaps", "parameters": {"year": "2023", "gp": "Monaco", "session_type": "Q", "driver1": "VER", "driver2": "LEC"}} \n\nNote: The tool will generate a lap speed comparison plot and provide the results, including who was faster.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-07-22T22:45:10.2875759Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1233624400, 'load_duration': 23580900, 'prompt_eval_count': 277, 'prompt_eval_duration': 2757300, 'eval_count': 70, 'eval_duration': 1206272300, 'model_name': 'llama3.1'}, id='run--484bc477-8027-456c-a004-4c39fae47239-0', usage_metadata={'input_tokens': 277, 'output_tokens': 70, 'total_tokens': 347})]}

In [8]:
result['messages'][-1].content  # This will contain the plot or the result of the tool call

'{"name": "CompareDriverLaps", "parameters": {"year": "2023", "gp": "Monaco", "session_type": "Q", "driver1": "VER", "driver2": "LEC"}}'